In [58]:
import json
import http.client
import time
# import boto3
from datetime import datetime
import pandas as pd

In [59]:
# the index for each dataframe will serve as the id number
location_df_base = pd.DataFrame(columns = ["state_id", "location_name", "location_lat", "location_long", "location_code"])
state_df_base = pd.DataFrame(columns = ["country_id", "state_name", "state_code"])
country_df_base = pd.DataFrame(columns = ["country_name", "country_code"])

In [60]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [61]:
# GLOBAL VARIBALE CONTAINING THE LAST ID NUMBER FOR EACH TABLE
country_last_id = 0
state_last_id = 0
location_last_id = 0
time_last_id = 0
forecast_last_id = 0
observation_last_id = 0
fact_last_id = 0

global_variables = pd.DataFrame({"table":["country", "state", "location", "time", "forecast", "obersvation", "fact"], "last_id":[country_last_id,  state_last_id, location_last_id, time_last_id, forecast_last_id, observation_last_id, fact_last_id]})
global_variables


,table,last_id
0,country,0
1,state,0
2,location,0
3,time,0
4,forecast,0
5,obersvation,0
6,fact,0


In [62]:
# Getting a list of all countries, this w
conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

conn.request("GET", "/v1/geo/countries?limit=10", headers=headers)

res = conn.getresponse()
data = res.read()
decodedData = data.decode("utf-8")
decodedData_json = json.loads(decodedData)

# print(decodedData_json["data"])

for city in decodedData_json["data"]:
    print(city)
    print()

country = {'code': 'AU', 'currencyCodes': ['AUD'], 'name': 'Australia', 'wikiDataId': 'Q408'}

country_df = pd.DataFrame({"country_name":country["name"], "country_code":country["code"]}, index=[country["wikiDataId"]])
country_df_base = pd.concat([country_df_base, country_df])

time.sleep(1)

{'code': 'AF', 'currencyCodes': ['AFN'], 'name': 'Afghanistan', 'wikiDataId': 'Q889'}

{'code': 'AL', 'currencyCodes': ['ALL'], 'name': 'Albania', 'wikiDataId': 'Q222'}

{'code': 'DZ', 'currencyCodes': ['DZD'], 'name': 'Algeria', 'wikiDataId': 'Q262'}

{'code': 'AD', 'currencyCodes': ['EUR'], 'name': 'Andorra', 'wikiDataId': 'Q228'}

{'code': 'AO', 'currencyCodes': ['AOA'], 'name': 'Angola', 'wikiDataId': 'Q916'}

{'code': 'AG', 'currencyCodes': ['XCD'], 'name': 'Antigua and Barbuda', 'wikiDataId': 'Q781'}

{'code': 'AR', 'currencyCodes': ['ARS'], 'name': 'Argentina', 'wikiDataId': 'Q414'}

{'code': 'AM', 'currencyCodes': ['AMD'], 'name': 'Armenia', 'wikiDataId': 'Q399'}

{'code': 'AU', 'currencyCodes': ['AUD'], 'name': 'Australia', 'wikiDataId': 'Q408'}

{'code': 'AT', 'currencyCodes': ['EUR'], 'name': 'Austria', 'wikiDataId': 'Q40'}



In [63]:
country_df_base

,country_name,country_code
Q408,Australia,AU


In [64]:
# Getting country details

conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

for i in range(country_df_base.shape[0]):
    conn.request("GET", "/v1/geo/countries/"+str(country_df_base.iloc[i]["country_code"]), headers=headers)

    res = conn.getresponse()
    data = res.read()
    decodedData = data.decode("utf-8")
    decodedData_json = json.loads(decodedData)

    print(decodedData_json["data"])

    time.sleep(1)

{'capital': 'Canberra', 'code': 'AU', 'callingCode': '+61', 'currencyCodes': ['AUD'], 'flagImageUri': 'http://commons.wikimedia.org/wiki/Special:FilePath/Flag%20of%20Australia%20%28converted%29.svg', 'name': 'Australia', 'numRegions': 11, 'wikiDataId': 'Q408'}


In [67]:
# Getting reigon
conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

limit = 10


for i in range(country_df_base.shape[0]):
    while (True):
        # conn.request("GET", "/v1/geo/countries/"+str(country_df_base.iloc[i]["country_code"])+"/regions", headers=headers)
        conn.request("GET", "/v1/geo/countries/"+str(country_df_base.iloc[i]["country_code"])+"/regions?offset="+str(offset)+"&limit="+str(limit), headers=headers)

        res = conn.getresponse()
        data = res.read()
        decodedData = data.decode("utf-8")
        decodedData_json = json.loads(decodedData)

        # print(decodedData_json)
        print(len(decodedData_json["data"]))

        if len(decodedData_json["data"]) == 0:
            break

        for reigon in decodedData_json["data"]:
            print(reigon)
            state_df = pd.DataFrame({"country_id":country_df_base.iloc[i].name, "state_name":reigon["name"], "state_code":reigon["isoCode"]}, index=[reigon["wikiDataId"]])
            state_df_base = pd.concat([state_df_base, state_df])

        offset += limit

        time.sleep(1.5)

10
{'countryCode': 'AU', 'fipsCode': '01', 'isoCode': 'ACT', 'name': 'Australian Capital Territory', 'wikiDataId': 'Q3258'}
{'countryCode': 'AU', 'fipsCode': 'CK', 'isoCode': 'CC', 'name': 'Cocos (Keeling) Islands', 'wikiDataId': 'Q36004'}
{'countryCode': 'AU', 'fipsCode': 'HM', 'isoCode': 'HM', 'name': 'Heard Island and McDonald Islands', 'wikiDataId': 'Q131198'}
{'countryCode': 'AU', 'fipsCode': '02', 'isoCode': 'NSW', 'name': 'New South Wales', 'wikiDataId': 'Q3224'}
{'countryCode': 'AU', 'fipsCode': 'NF', 'isoCode': 'NF', 'name': 'Norfolk Island', 'wikiDataId': 'Q31057'}
{'countryCode': 'AU', 'fipsCode': '03', 'isoCode': 'NT', 'name': 'Northern Territory', 'wikiDataId': 'Q3235'}
{'countryCode': 'AU', 'fipsCode': '04', 'isoCode': 'QLD', 'name': 'Queensland', 'wikiDataId': 'Q36074'}
{'countryCode': 'AU', 'fipsCode': '05', 'isoCode': 'SA', 'name': 'South Australia', 'wikiDataId': 'Q35715'}
{'countryCode': 'AU', 'fipsCode': '06', 'isoCode': 'TAS', 'name': 'Tasmania', 'wikiDataId': 'Q34

In [68]:
state_df_base

,country_id,state_name,state_code
Q3258,Q408,Australian Capital Territory,ACT
Q36004,Q408,Cocos (Keeling) Islands,CC
Q131198,Q408,Heard Island and McDonald Islands,HM
Q3224,Q408,New South Wales,NSW
Q31057,Q408,Norfolk Island,NF
Q3235,Q408,Northern Territory,NT
Q36074,Q408,Queensland,QLD
Q35715,Q408,South Australia,SA
Q34366,Q408,Tasmania,TAS
Q36687,Q408,Victoria,VIC


In [86]:
import http.client

conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

for j in range(state_df_base.shape[0]):
    print(state_df_base.iloc[j])

    conn.request("GET", "/v1/geo/countries/"+str(country_df_base.loc[state_df_base.iloc[j]["country_id"]]["country_code"])+"/regions/"+str(state_df_base.iloc[j]["state_code"]), headers=headers)
    
    res = conn.getresponse()
    data = res.read()
    decodedData = data.decode("utf-8")
    decodedData_json = json.loads(decodedData)

    # print(decodedData_json)
    print(len(decodedData_json["data"]))

    print(decodedData_json["data"])
    time.sleep(1.5)

    # this can be combined with the request above

country_id                            Q408
state_name    Australian Capital Territory
state_code                             ACT
Name: Q3258, dtype: object
8
{'capital': 'Canberra', 'countryCode': 'AU', 'fipsCode': '01', 'isoCode': 'ACT', 'name': 'Australian Capital Territory', 'numCities': 124, 'numPlaces': 130, 'wikiDataId': 'Q3258'}
country_id                       Q408
state_name    Cocos (Keeling) Islands
state_code                         CC
Name: Q36004, dtype: object
8
{'capital': 'West Island', 'countryCode': 'AU', 'fipsCode': 'CK', 'isoCode': 'CC', 'name': 'Cocos (Keeling) Islands', 'numCities': 1, 'numPlaces': 27, 'wikiDataId': 'Q36004'}
country_id                                 Q408
state_name    Heard Island and McDonald Islands
state_code                                   HM
Name: Q131198, dtype: object
7
{'countryCode': 'AU', 'fipsCode': 'HM', 'isoCode': 'HM', 'name': 'Heard Island and McDonald Islands', 'numCities': 0, 'numPlaces': 0, 'wikiDataId': 'Q131198'}
country_i

In [23]:
import http.client

conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

conn.request("GET", "/v1/geo/countries/AU/regions/NSW/cities", headers=headers)

res = conn.getresponse()
data = res.read()

decodedData = data.decode("utf-8")
decodedData_json = json.loads(decodedData)

print(decodedData_json["data"])

time.sleep(1)

KeyError: 'data'

In [96]:
import http.client

conn = http.client.HTTPSConnection("wft-geo-db.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "4be2272798mshecd2e852194b761p1ea3c8jsn77834570a147",
    'X-RapidAPI-Host': "wft-geo-db.p.rapidapi.com"
}

# conn.request("GET", "/v1/geo/countries/AU/regions/NSW/adminDivisions", headers=headers)


limit = 10

# for k in range(state_df_base.shape[0]):
for k in [3]: #range(state_df_base.shape[0]):
    offset = 0

    print("state: ", state_df_base.iloc[k]["state_code"])
    while (True): 
        time.sleep(1.5) 
        # conn.request("GET", "/v1/geo/countries/"+str(country_df_base.loc[state_df_base.iloc[k]["country_id"]]["country_code"])+"/regions/"+str(state_df_base.iloc[k]["state_code"])+"/adminDivisions?offset="+str(offset)+"&limit="+str(limit), headers=headers)
        conn.request("GET", "/v1/geo/countries/"+str(country_df_base.loc[state_df_base.iloc[k]["country_id"]]["country_code"])+"/regions/"+str(state_df_base.iloc[k]["state_code"])+"/cities?offset="+str(offset)+"&limit="+str(limit), headers=headers)

        res = conn.getresponse()
        data = res.read()

        decodedData = data.decode("utf-8")
        decodedData_json = json.loads(decodedData)

        if len(decodedData_json["data"]) == 0:
            break

        for city in decodedData_json["data"]:
            if city["population"] > 2000:
                print(city)
                location_df = pd.DataFrame({"state_id":state_df_base.iloc[k].name, "location_name":city["name"], "location_lat":city["latitude"], "location_long":city["longitude"]}, index=[city["wikiDataId"]])
                location_df_base = pd.concat([location_df_base, location_df])

        offset += limit

          

state:  NSW
{'id': 988751, 'wikiDataId': 'Q4664280', 'type': 'CITY', 'city': 'Abbotsbury', 'name': 'Abbotsbury', 'latitude': -33.873611111, 'longitude': 150.8594444, 'population': 4200}
{'id': 987701, 'wikiDataId': 'Q4664298', 'type': 'CITY', 'city': 'Abbotsford', 'name': 'Abbotsford', 'latitude': -33.856666666, 'longitude': 151.1261111, 'population': 5431}
{'id': 987920, 'wikiDataId': 'Q4666845', 'type': 'CITY', 'city': 'Aberdare', 'name': 'Aberdare', 'latitude': -32.842777777, 'longitude': 151.3802778, 'population': 2542}
{'id': 988049, 'wikiDataId': 'Q2821571', 'type': 'CITY', 'city': 'Aberdeen', 'name': 'Aberdeen', 'latitude': -32.165, 'longitude': 150.9011111, 'population': 2051}
{'id': 987740, 'wikiDataId': 'Q4667103', 'type': 'CITY', 'city': 'Aberglasslyn', 'name': 'Aberglasslyn', 'latitude': -32.698333333, 'longitude': 151.5344444, 'population': 6552}
{'id': 987784, 'wikiDataId': 'Q4667131', 'type': 'CITY', 'city': 'Abermain', 'name': 'Abermain', 'latitude': -32.807222222, 'lon

KeyError: 'data'